In [10]:
import json
import pickle
import networkx as nx
import pandas as pd
from pyvis.network import Network

# Función para cargar archivos JSON
def load_json(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

# Función para cargar archivos pickle
def load_pickle(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

# Cargar datos
data_autores = load_json('data/grafo_autores.json')
data_papers = load_json('data/lista_papers.json')
data_inst = load_json('data/grafo_instituciones.json')
data_paper_inst = load_json('data/lista_papers_instituciones.json')
data_cluster = load_pickle("data/cluster_authorrank_2000.pkl")
data_cluster_filiacion = load_pickle("data/cluster_filiacion_authorrank_500.pkl")

# Cargar archivos Excel
salida_articulos = pd.read_excel("data/output_articles.xlsx")
salidas_topicos = pd.read_excel("data/salidas_topicos_CB.xlsx")
salida_autores = pd.read_excel("data/salida_autores.xlsx")

# Convertir los valores de nodos a str para que el código funcione sin modificaciones
data_papers = {key: [str(node) for node in value] for key, value in data_papers.items()}

In [11]:
# La enumeracion del cluster va en orden descendente de acuerdo al 
# orden de los grandes productores

def enumerar_clusters(data_cluster):
    """
    Enumerar los clusters a partir de la data_cluster.
    
    Parameters:
    data_cluster (list): Lista de clusters donde cada elemento es un string con nodos separados por '_'.
    
    Returns:
    dict: Diccionario con clusters enumerados.
    """
    enumerar_cluster = {}
    for i, cluster in enumerate(data_cluster, start=1):
        enumerar_cluster[i] = cluster.split('_')
    return enumerar_cluster

def enumerar_nodos(enumerar_cluster):
    """
    Crear un diccionario que asocia cada nodo a su cluster correspondiente.
    
    Parameters:
    enumerar_cluster (dict): Diccionario de clusters enumerados.
    
    Returns:
    dict: Diccionario que mapea cada nodo a su cluster.
    """
    enumerar_nodo = {}
    for cluster_id, nodos in enumerar_cluster.items():
        for nodo in nodos:
            enumerar_nodo[nodo] = cluster_id
    return enumerar_nodo

Enumerar_cluster = enumerar_clusters(data_cluster)

enumerar_nodo = enumerar_nodos(Enumerar_cluster)


In [12]:
def crear_grafo_comunidades(enumerar_cluster, data_autores, num_comunidades=16, min_articulos=5):
    """
    Crear un grafo de comunidades basado en coautorías.

    Parameters:
    enumerar_cluster (dict): Diccionario de clusters enumerados.
    data_autores (dict): Diccionario con datos de autores.
    num_comunidades (int): Número de comunidades a incluir.
    min_articulos (int): Número mínimo de artículos para incluir un nodo.

    Returns:
    G (networkx.Graph): Grafo de comunidades.
    node_color (dict): Diccionario que mapea cada nodo a su color.
    """
    colores = [
        'red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink', 'brown',
        'black', 'white', 'gray', 'cyan', 'magenta', 'lime', 'maroon', 'navy'
    ]

    if num_comunidades > len(colores):
        raise ValueError("El número de comunidades supera el número de colores disponibles.")

    G = nx.Graph()
    node_color = {}

    for i in range(1, num_comunidades + 1):
        for node in enumerar_cluster.get(i, []):
            if data_autores[node]['n_articulos'] >= min_articulos:
                G.add_node(node)
                node_color[node] = colores[i - 1]

    for node in G.nodes():
        for coautor in data_autores[node].get("coautores", []):
            if coautor in G:
                G.add_edge(node, coautor)

    return G, node_color

n_comn = 16

G, node_color = crear_grafo_comunidades(Enumerar_cluster, data_autores, num_comunidades=n_comn)

print(f'Nodos: {G.number_of_nodes()}')
print(f'Aristas: {G.number_of_edges()}')

Nodos: 92
Aristas: 380


In [13]:
def crear_red_comunidades(G, data_autores, height="750px", width="100%", bgcolor="#222222", font_color="white"):
    """
    Crear una red de comunidades utilizando pyvis y datos de coautorías.

    Parameters:
    G (networkx.Graph): Grafo de comunidades.
    data_autores (dict): Diccionario con datos de autores.
    height (str): Altura del gráfico.
    width (str): Ancho del gráfico.
    bgcolor (str): Color de fondo del gráfico.
    font_color (str): Color de fuente del gráfico.

    Returns:
    Network: Objeto de red de pyvis.
    """
    net = Network(height=height, width=width, bgcolor=bgcolor, font_color=font_color, notebook=True, cdn_resources='in_line')

    # Añadir nodos y aristas a la red
    for edge in G.edges():
        src, dst = map(str, edge)
        n_art = data_autores[edge[0]]['coautores'].get(edge[1], 0)

        net.add_node(src, src, title=src)
        net.add_node(dst, dst, title=dst)
        if n_art != 0:
            net.add_edge(src, dst, value=n_art)

    return net

# Crear la red de comunidades
comn_20_net = crear_red_comunidades(G, data_autores)

# Mostrar información de la red (opcional)
print(f'Nodos en la red: {len(comn_20_net.nodes)}')
print(f'Aristas en la red: {len(comn_20_net.edges)}')

Nodos en la red: 92
Aristas en la red: 380


In [14]:
def actualizar_nodos_red(net, data_autores, node_color):
    """
    Actualizar los nodos de la red con información adicional.

    Parameters:
    net (Network): Objeto de red de pyvis.
    data_autores (dict): Diccionario con datos de autores.
    node_color (dict): Diccionario que mapea cada nodo a su color.
    """
    for node in net.nodes:
        node_id = node["id"]
        n_art = data_autores[node_id]["n_articulos"]

        node["label"] = data_autores[node_id]['nombre_autor']
        node["value"] = 2 * n_art
        node["font"] = {'size': 2 * n_art, 'color': "white"}
        node["title"] = f"Número de artículos: {n_art}"
        node["color"] = node_color[node_id]

def mostrar_red_con_botones(net, filter_=["physics"]):
    """
    Mostrar la red con botones de filtro.

    Parameters:
    net (Network): Objeto de red de pyvis.
    filter_ (list): Lista de filtros para los botones de pyvis.
    """
    net.show_buttons(filter_=filter_)

# Actualizar nodos de la red con información adicional
actualizar_nodos_red(comn_20_net, data_autores, node_color)

# # Mostrar la red con botones de filtro
# mostrar_red_con_botones(comn_20_net)

In [15]:
comn_20_net.show('Grafo20Comunidad.html') 

Grafo20Comunidad.html
